In [ ]:
import jsonlines
import re
import json
import os

In [ ]:
with jsonlines.open('xx/ori/CommonsenseQA/dev_rand_split.jsonl', 'r') as f:
    gold_datas = [data for data in f]

In [ ]:
gold_options = []
gold_dict = {}
for index,data in enumerate(gold_datas):
    question = data['question']['stem']
    gold_dict[question] = index
    gold_options.append(data['question']['choices'])

In [ ]:
gold_tuples = []
for index, data in enumerate(gold_datas):
    answerKey = data['answerKey']
    answerKey_index = ord(answerKey)-ord('A')
    answer_choice = data['question']['choices'][answerKey_index]
    assert answer_choice['label'] == answerKey
    gold_tuples.append((index, answer_choice['text']))

In [ ]:
def calc_acc(pred_tuples, gold_tuples):
    assert len(pred_tuples) == len(gold_tuples)
    pred_tuples = set(pred_tuples)
    gold_tuples = set(gold_tuples)
    corr_tuples = pred_tuples & gold_tuples
    try:
        acc = len(corr_tuples) / len(pred_tuples)
    except:
        acc = 0
    return acc

In [ ]:
read_files = [
'c_to_hrt_dev_Commonsense.jsonl',
'yes_no_dev_Commonsense.jsonl',
'fewer_option_dev_Commonsense.jsonl',
'all_fewer_option_dev_Commonsense.jsonl'
]
read_dir = './pred_result'

In [ ]:
for sub_dir in os.listdir(read_dir):
    
    for read_file in read_files:
        with jsonlines.open(os.path.join(read_dir, sub_dir, read_file), 'r') as f:
            pred_datas = [data for data in f]
        if 'c_to_hrt' in read_file:
            pred_tuples = []
            for index,data in enumerate(pred_datas):
                ans = data['output'].strip()
                answerKey_index = ord(ans)-ord('A')
                pred_tuples.append((index, gold_options[index][answerKey_index]['text']))
            acc = calc_acc(pred_tuples, gold_tuples)
            print('pred_file:{}'.format(os.path.join(read_dir, sub_dir, read_file)))
            print('acc:{}\n'.format(round(acc*100,2)))
        if 'yes_no' in read_file:
            pred_tuples = []
            for index,data in enumerate(pred_datas):
                text = data['text']
                options = data['options']
                is_finish = data['is_finish']
                if is_finish:
                    assert len(options) == 1
                    ori_index = gold_dict[text]
                    pred_tuples.append((ori_index, options[0]))
        elif '/fewer_option' in read_file:
            for data in pred_datas:
                text = data['text']
                options = data['options']
                pred = data['output'].strip()
                try:
                    ans = options[ord(pred)-ord('A')]
                except:
                    ans = ''
                ori_index = gold_dict[text]
                pred_tuples.append((ori_index, ans))
            acc = calc_acc(pred_tuples, gold_tuples)
            print('pred_file:{}'.format(os.path.join(read_dir, sub_dir, read_file)))
            print('acc:{}\n'.format(round(acc*100,2)))
        elif 'all_fewer_option_dev_Commonsense' in read_file:
            pred_tuples = []
            text_option_count = {}
            for data in pred_datas:
                text = data['text']
                options = data['options']
                if text not in text_option_count.keys():
                    text_option_count[text] = {}
                for option in options:
                    if option not in text_option_count[text].keys():
                        text_option_count[text][option] = 0
                output = data['output']
                try:
                    ans = options[ord(output.strip()) - ord('A')]
                except:
                    ans = options[0]
                
                text_option_count[text][ans] += 1
                text_option_count[text]['_'.join(list(sorted(options)))] = ans
            for question, index in gold_dict.items():
                now_text_option_count = text_option_count[question]
                # Remove the special key that stores the sorted options
                counts = {k: v for k, v in now_text_option_count.items() if '_' not in k}
                # assert len(counts.keys()) == 5
                
                # Find the maximum count
                max_count = max(counts.values())
                
                # Get all options with max count
                candidates = [k for k, v in counts.items() if v == max_count]
                
                if len(candidates) == 1:
                    ans = candidates[0]
                else:
                    # If there's a tie, use the answer stored with sorted options
                    sorted_options_key = '_'.join(list(sorted(candidates)))
                    ans = now_text_option_count[sorted_options_key]
                pred_tuples.append((index, ans))
            acc = calc_acc(pred_tuples, gold_tuples)
            print('pred_file:{}'.format(os.path.join(read_dir, sub_dir, read_file)))
            print('acc:{}\n'.format(round(acc*100,2)))

In [ ]:
text_option_count

In [ ]:
candidates

In [ ]:
data